In [1]:
from netCDF4 import Dataset
from netCDF4 import num2date
import matplotlib
matplotlib.use('Agg')  # Use the 'Agg' backend which doesn't require a display
#matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import pandas as pd
import xarray as xr
import scipy
from scipy import special
from scipy.stats import norm
from scipy.stats import ks_2samp as ks_2samp
import sys
import cdo
from cdo import *   # python version
cdo = Cdo()
import os
from glob import glob
import cftime
from scipy.spatial import cKDTree
from cartopy.mpl.gridliner import LongitudeFormatter, LatitudeFormatter
import seaborn as sns

MODELS = [ 'BCCR-AUTH','BTU','CMCC','CNRM','ETHZ','FZJ-IDL','HCLIM','ICTP','KIT','KNMI','UKMO']
MODELS = [ 'BCCR-AUTH','BTU','CMCC','CNRM','ETHZ','FZJ-IDL','HCLIM','ICTP','KIT','KNMI','UKMO','JLU']

filename_nonCP = [ \
f'tasmax_EUR-15_NorESM1-ME_historical_r1i1p1_BCCR-WRF381DA_fpsconv-x1n2-v1_day_JJA.nc', \
f'tasmax_EUR-11_CNRM-CERFACS-CNRM-CM5_historical_r1i1p1_CLMcom-CCLM4-8-17_cordex-x0n1-v1_day_JJA.nc', \
f'tasmax_EUR-11_ICHEC-EC-EARTH_historical_r12i1p1_CLMcom-CMCC-CCLM5-0-9_v1_day_JJA.nc', \
f'tasmax_EUR-11_CNRM-CERFACS-CNRM-CM5_historical_r1i1p1_CNRM-ALADIN63_v2_day_JJA.nc', \
f'tasmax_EUR-11_MPI_historical_r1i1p1_COSMO-pompa_5.0_2019.1_day_JJA.nc', \
f'tasmax_EUR-15_SMHI-EC-EARTH_historical_r12_FZJ-IBG3-WRF381CA_v1_day_JJA.nc', \
f'tasmax_ALP-12_ICHEC-EC-EARTH_historical_r12i1p1_HCLIMcom-HCLIM38-ALADIN_v1_day_JJA.nc', \
f'tasmax_EUR-11_MOHC-HadGEM2-ES_historical_r1i1p1_ICTP-RegCM4-7_fpsconv-x2yn1-v1_day_JJA.nc', \
f'tasmax_EUR-11_MPI-M-MPI-ESM-LR_historical_r1i1p1_CLMcom-CCLM4-8-17_v1_day_JJA.nc', \
f'tasmax_WCE-11_KNMI-EC-EARTH_historical_r14i1p1_KNMI-RACMO23E_v1_day_JJA.nc', \
f'tasmax_EUR-11_MOHC-HadGEM2-ES_historical_r1i1p1_MOHC-HadREM3-GA7-05_v1_day_JJA.nc', \
]

filename_CP = [ \
'tasmax_ALP-3_NorESM1-ME_historical_r1i1p1_BCCR-WRF381DA_fpsconv-x1n2-v1_day_JJA.nc',\
'tasmax_ALP-3_CNRM-CERFACS-CNRM-CM5_historical_r1i1p1_CLMcom-BTU-CCLM5-0-14_fpsconv-x2yn2-v1_day_JJA.nc', \
'tasmax_ALP-3_ICHEC-EC-EARTH_historical_r12i1p1_CLMcom-CMCC-CCLM5-0-9_x2yn2v1_day_JJA.nc', \
'tasmax_ALP-3_CNRM-CERFACS-CNRM-CM5_historical_r1i1p1_CNRM-AROME41t1_fpsconv-x2yn2-v1_day_JJA.nc', \
'tasmax_ALP-3_MPI_historical_r1i1p1_COSMO-pompa_5.0_2019.1_day_JJA.nc', \
'tasmax_ALP-3_SMHI-EC-EARTH_historical_r12_FZJ-IBG3-WRF381CA_v1_day_JJA.nc', \
'tasmax_ALP-3_ICHEC-EC-EARTH_historical_r12i1p1_HCLIMcom-HCLIM38-AROME_fpsconv-x2yn2-v1_day_JJA.nc', \
'tasmax_ALP-3_MOHC-HadGEM2-ES_historical_r1i1p1_ICTP-RegCM4-7_fpsconv-x2yn2-v1_day_JJA.nc', \
'tasmax_ALP-3_MPI-M-MPI-ESM-LR_historical_r1i1p1_CLMcom-KIT-CCLM5-0-15_fpsconv-x2yn2-v1_day_JJA.nc', \
'tasmax_ALP-3_KNMI-EC-EARTH_historical_r14i1p1_KNMI-HCLIM38h1-AROME_fpsconv-x2yn2-v1_day_JJA.nc', \
'tasmax_ALP-3_HadGEM3-GC3.1-N512_historical_r1i1p1_HadREM3-RA-UM10.1_fpsconv-x0n1-v1_day_JJA.nc', \
'tasmax_ALP-3_MPI-M-MPI-ESM-LR_historical_r1i1p1_CLMcom-JLU-CCLM5-0-15_fpsconv-x0n1-v1_day_JJA.nc'\
]


### Load and plot observations

In [2]:
OBS_DIR = '/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2/'

# Initialize an empty list to store DataFrames for each file
all_dataframes = []
count_nan_per_site = []
mask_nan=[]

# Specify the latitude and longitude range
lat_range = slice(40, 50)  # Adjust as needed
lon_range = slice(0, 20)  # Adjust as needed

# Define the time range for the years 1996 to 2005
start_date = '1996-01-01'
end_date = '2005-12-31'
        
# Create an empty list to store DataFrames for each site
site_dfs = []

# Loop through each file in the directory
for filename in os.listdir(f'{OBS_DIR}'):
    if filename.endswith('.nc'):
        # Construct the full path to the file
        file_path = os.path.join(f'{OBS_DIR}/{filename}')
        
        print(f'{file_path}')
        
        # Open the NetCDF file
        ds = xr.open_dataset(f'{file_path}')
        
        # Extract lon, lat, and elevation (assuming these variables exist in the dataset)
        lon = ds['longitude'].values.item()
        lat = ds['latitude'].values.item()
        elevation = ds['elevation'].values.item()

        # Extract temperature variable
        temperature = ds['temperatures']
        time = ds['time']
        
        if lon_range.start <= lon <= lon_range.stop and lat_range.start <= lat <= lat_range.stop:
            
            # Select the temperature data for the specified time range
            selected_temperature = temperature.sel(time=slice(start_date, end_date))
            selected_time = time.sel(time=slice(start_date, end_date))
            
            # Create a complete hourly calendar for the specified time range
            complete_calendar = pd.date_range(start=start_date, end=end_date, freq='H')
            
            # Identify available time steps and corresponding temperature values
            available_time_steps = pd.to_datetime(selected_time.values)
            available_temperature_values = selected_temperature.values
            
            # Create a DataFrame with the complete calendar
            df_complete_calendar = pd.DataFrame({'time': complete_calendar})
            
            # Merge the complete calendar with the available time steps and temperature values
            df_merged = pd.merge(df_complete_calendar, pd.DataFrame({'time': available_time_steps, 'temperature': available_temperature_values}),
                     on='time', how='left')
            
            # Assuming df_merged has 'time' as a column, convert it to datetime and set it as the index
            df_merged['time'] = pd.to_datetime(df_merged['time'])
            df_merged.set_index('time', inplace=True)

            # Resample to daily frequency and calculate daily mean temperature
            # Set min_count to 12 to require at least 12 non-null values for each day
            #df_daily_max = df_merged.resample('D').max(min_count=12)

            df_daily_max = df_merged.groupby(pd.Grouper(freq='D')).max(min_count=12)
            
            # Reset the index to get 'time' back as a regular column
            df_daily_max.reset_index(inplace=True)
    
            df_daily_max_jja = df_daily_max[(df_daily_max['time'].dt.month >= 6) & (df_daily_max['time'].dt.month <= 8)]
    
            # Add additional information to the DataFrame for the first time step only
            df_daily_max_jja.loc[df_daily_max_jja.index[0], 'lon'] = lon
            df_daily_max_jja.loc[df_daily_max_jja.index[0], 'lat'] = lat
            df_daily_max_jja.loc[df_daily_max_jja.index[0], 'elevation'] = elevation
            df_daily_max_jja.loc[df_daily_max_jja.index[0], 'filename'] = filename
            
            # Append the DataFrame to the list
            all_dataframes.append(df_daily_max_jja)


/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202309p_19310101-20231001_010014-99999.nc
/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202309p_19310101-20231001_011020-99999.nc
/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202309p_19310101-20231001_011025-99999.nc
/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202309p_19310101-20231001_011120-99999.nc
/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202309p_19310101-20231001_011160-99999.nc
/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202309p_19310101-20231001_011220-99999.nc
/work3/lorenzosangelantoni/s

/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202309p_19310101-20231001_013850-99999.nc
/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202309p_19310101-20231001_013890-99999.nc
/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202309p_19310101-20231001_013920-99999.nc
/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202309p_19310101-20231001_013970-99999.nc
/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202309p_19310101-20231001_014000-99999.nc
/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202309p_19310101-20231001_014030-99999.nc
/work3/lorenzosangelantoni/s

/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202309p_19310101-20231001_022420-99999.nc
/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202309p_19310101-20231001_022430-99999.nc
/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202309p_19310101-20231001_022450-99999.nc
/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202309p_19310101-20231001_022470-99999.nc
/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202309p_19310101-20231001_022540-99999.nc
/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202309p_19310101-20231001_022590-99999.nc
/work3/lorenzosangelantoni/s

/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202309p_19310101-20231001_024500-99999.nc
/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202309p_19310101-20231001_024520-99999.nc
/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202309p_19310101-20231001_024530-99999.nc
/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202309p_19310101-20231001_024560-99999.nc
/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202309p_19310101-20231001_024580-99999.nc
/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202309p_19310101-20231001_024600-99999.nc
/work3/lorenzosangelantoni/s

/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202309p_19310101-20231001_025625-99999.nc
/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202309p_19310101-20231001_025630-99999.nc
/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202309p_19310101-20231001_025635-99999.nc
/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202309p_19310101-20231001_025650-99999.nc
/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202309p_19310101-20231001_025660-99999.nc
/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202309p_19310101-20231001_025700-99999.nc
/work3/lorenzosangelantoni/s

...

/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202310p_19310101-20231101_165200-99999.nc
/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202310p_19310101-20231101_165220-99999.nc
/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202310p_19310101-20231101_165310-99999.nc


/tmp/ipykernel_68343/1604952351.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_daily_max_jja.loc[df_daily_max_jja.index[0], 'lon'] = lon
/tmp/ipykernel_68343/1604952351.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_daily_max_jja.loc[df_daily_max_jja.index[0], 'lat'] = lat
/tmp/ipykernel_68343/1604952351.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202310p_19310101-20231101_165380-99999.nc
/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202310p_19310101-20231101_165390-99999.nc
/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202310p_19310101-20231101_165410-99999.nc
/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202310p_19310101-20231101_165420-99999.nc
/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202310p_19310101-20231101_165460-99999.nc
/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202310p_19310101-20231101_165500-99999.nc
/work3/lorenzosangelantoni/s

/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202310p_19310101-20231101_607400-99999.nc
/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202310p_19310101-20231101_607403-99999.nc
/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202310p_19310101-20231101_607480-99999.nc
/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202310p_19310101-20231101_990180-99999.nc
/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/scripts/python_2/evaluation/outputs2//hadisd.3.3.1.202310p_19310101-20231101_999999-34048.nc


/tmp/ipykernel_68343/1604952351.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_daily_max_jja.loc[df_daily_max_jja.index[0], 'lon'] = lon
/tmp/ipykernel_68343/1604952351.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_daily_max_jja.loc[df_daily_max_jja.index[0], 'lat'] = lat
/tmp/ipykernel_68343/1604952351.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

### Plot observation sites

In [3]:
def filter_inconsistent_values(df):
    # Replace temperature values outside the range with NaN
    df['temperature'] = np.where((df['temperature'] < -100) | (df['temperature'] > 100), np.nan, df['temperature'])
    return df

# Apply the filtering to each DataFrame in all_dataframes
filtered_dataframes = [filter_inconsistent_values(df.copy()) for df in all_dataframes]


# Extract mean Tmax values and site coordinates
mean_tmax_values = [df['temperature'].mean() for df in filtered_dataframes]
site_lons = [df['lon'].iloc[0] if not df.empty else None for df in filtered_dataframes]
site_lats = [df['lat'].iloc[0] if not df.empty else None for df in filtered_dataframes]

# Remove None values from site_lons and site_lats
#mean_tmax_values = [mean_tmax for mean_tmax, lon, lat in zip(mean_tmax_values, site_lons, site_lats) if lon is not None and lat is not None]
#site_lons = [lon for lon in site_lons if lon is not None]
#site_lats = [lat for lat in site_lats if lat is not None]

# Create a map using Cartopy
fig, ax = plt.subplots(subplot_kw={'projection': ccrs.PlateCarree()})
ax.coastlines()
# Add land-sea mask first
land_mask = cfeature.NaturalEarthFeature('physical', 'land', '10m', edgecolor='face', facecolor=cfeature.COLORS['land'])
ax.add_feature(land_mask, zorder=0)  # Set zorder to 0 to ensure it's plotted below the scatter points
ax.set_extent([lon_range.start, 17, lat_range.start, lat_range.stop])  # Limit to 17 degrees west
 
# Add land and state borders
ax.add_feature(cfeature.BORDERS, linestyle='-')
ax.add_feature(cfeature.COASTLINE)

# Add lon/lat gridlines
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=1, color='gray', alpha=0.5, linestyle='-')
gl.xlabels_top = False
gl.ylabels_right = False
gl.xformatter = LongitudeFormatter(degree_symbol='°', dateline_direction_label=True)
gl.yformatter = LatitudeFormatter(degree_symbol='°')
# Set gridline intervals to every 2 degrees
gl.xlocator = plt.MaxNLocator(5)
gl.ylocator = plt.MaxNLocator(5)

  # Scatter plot of sites with mean Tmax values as colors
sc = ax.scatter(site_lons, site_lats, c=mean_tmax_values, cmap='viridis', s=30, edgecolors='k', vmin=min(mean_tmax_values), vmax=max(mean_tmax_values))

# Add colorbar
cbar = plt.colorbar(sc, ax=ax, label='Mean Tmax')
cbar.set_label('Mean Tmax', rotation=270, labelpad=15)

# Add title and labels
plt.title('Sites with Mean Tmax (Filtered)')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Show the plot
plt.savefig('reference_sites.png')

/home/lorenzosangelantoni/.conda/envs/DEVELOP/lib/python3.10/site-packages/cartopy/mpl/gridliner.py:451: UserWarning: The .xlabels_top attribute is deprecated. Please use .top_labels to toggle visibility instead.
  warnings.warn('The .xlabels_top attribute is deprecated. Please '
/home/lorenzosangelantoni/.conda/envs/DEVELOP/lib/python3.10/site-packages/cartopy/mpl/gridliner.py:487: UserWarning: The .ylabels_right attribute is deprecated. Please use .right_labels to toggle visibility instead.
  warnings.warn('The .ylabels_right attribute is deprecated. Please '


### DAV

In [10]:
# Initialize lists to store values for each model
all_S_nonCP = []
all_S_CP = []
all_DAV = []
model_names = []

# Initialize DataFrames to store values for each model
df_S_nonCP = pd.DataFrame(index=range(0,len(site_lons)), columns=MODELS)
df_S_CP = pd.DataFrame(index=range(0,len(site_lons)), columns=MODELS)
df_DAV = pd.DataFrame(index=range(0,len(site_lons)), columns=MODELS)

SIM_DIR_nonCP = '/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/FILES_nonCP/tasmax/historical'
SIM_DIR_CP = '/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/FILES_CP/tasmax/historical'

for i in range(0,len(MODELS)) : 
    
    
    if i == len(MODELS)-1: 
        file_path = os.path.join(f'{SIM_DIR_nonCP}/{filename_nonCP[8]}')
        print(f'{MODELS[8]} nonCP (II)')
    else : 
        file_path = os.path.join(f'{SIM_DIR_nonCP}/{filename_nonCP[i]}')
        print(f'{MODELS[i]} nonCP')
        
    nc = Dataset(file_path)
    lon_model = nc.variables['lon'][:]
    lat_model = nc.variables['lat'][:]
    tasmax = nc.variables['tasmax'][:]
    
    # Reference site coordinates
    reference_coords = list(zip(site_lons, site_lats))
    
    # Flatten the lat and lon arrays from the model data
    model_lon_flat = lon_model.flatten()
    model_lat_flat = lat_model.flatten()

    # Create a KDTree from the flattened lon and lat arrays
    tree = cKDTree(list(zip(model_lon_flat, model_lat_flat)))

    # Query the KDTree to find the nearest neighbors for each reference site
    nearest_indices = []

    for lon, lat in reference_coords:
        dist, idx = tree.query([lon, lat])
        
        # Check if the index is within bounds
        if 0 <= idx < len(model_lon_flat):
            nearest_indices.append(idx)
        else:
            # Handle the case when the index is out of bounds
            nearest_indices.append(np.nan)

    # Extract the lon and lat values for the nearest grid nodes
    nearest_lon = [model_lon_flat[int(idx)] if not np.isnan(idx) else np.nan for idx in nearest_indices]
    nearest_lat = [model_lat_flat[int(idx)] if not np.isnan(idx) else np.nan for idx in nearest_indices]

    # Reshape the tasmax_data variable to have a flattened lat and lon dimension
    tasmax_data_flat = tasmax.reshape(tasmax.shape[0], -1)

    # Initialize an empty list to store tasmax values for each reference site
    tasmax_values = []

    # Extract tasmax values for each reference site
    for idx in nearest_indices:
        if not np.isnan(idx):
            tasmax_values.append(tasmax_data_flat[:, int(idx)])
        else:
            # Handle the case when the index is NaN
            tasmax_values.append(np.full(tasmax.shape[0], np.nan))

    # Convert the list of tasmax values to a NumPy array
    tasmax_values_nonCP = np.array(tasmax_values)-273.16

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
    
    del(tasmax_values)
    
    file_path = os.path.join(f'{SIM_DIR_CP}/{filename_CP[i]}')
        
    print(f'{MODELS[i]} CP')
        
    nc = Dataset(file_path)
    lon_model = nc.variables['lon'][:]
    lat_model = nc.variables['lat'][:]
    tasmax = nc.variables['tasmax'][:]
    
    # Reference site coordinates
    reference_coords = list(zip(site_lons, site_lats))
    
    # Flatten the lat and lon arrays from the model data
    model_lon_flat = lon_model.flatten()
    model_lat_flat = lat_model.flatten()

    # Create a KDTree from the flattened lon and lat arrays
    tree = cKDTree(list(zip(model_lon_flat, model_lat_flat)))

    # Query the KDTree to find the nearest neighbors for each reference site
    nearest_indices = []

    for lon, lat in reference_coords:
        dist, idx = tree.query([lon, lat])
        
        # Check if the index is within bounds
        if 0 <= idx < len(model_lon_flat):
            nearest_indices.append(idx)
        else:
            # Handle the case when the index is out of bounds
            nearest_indices.append(np.nan)

    # Extract the lon and lat values for the nearest grid nodes
    nearest_lon = [model_lon_flat[int(idx)] if not np.isnan(idx) else np.nan for idx in nearest_indices]
    nearest_lat = [model_lat_flat[int(idx)] if not np.isnan(idx) else np.nan for idx in nearest_indices]

    # Reshape the tasmax_data variable to have a flattened lat and lon dimension
    tasmax_data_flat = tasmax.reshape(tasmax.shape[0], -1)

    # Initialize an empty list to store tasmax values for each reference site
    tasmax_values = []

    # Extract tasmax values for each reference site
    for idx in nearest_indices:
        if not np.isnan(idx):
            tasmax_values.append(tasmax_data_flat[:, int(idx)])
        else:
            # Handle the case when the index is NaN
            tasmax_values.append(np.full(tasmax.shape[0], np.nan))

    # Convert the list of tasmax values to a NumPy array
    tasmax_values_CP = np.array(tasmax_values)-273.16

    
# # # #  # # # # # # # # # # # # # # # # # # # # # DAV # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

    DAV_sites=[]

    # Calculate the ECDF
    def ecdf(data):
        sorted_data = np.sort(data)
        n = len(data)
        y = np.arange(1, n + 1) / n
        return sorted_data, y
    
    # Assuming you have a counter defined somewhere
    counter = 0
    for j in range(0,len(site_lons)) : 

        observed_data = filtered_dataframes[j]['temperature']
        percentage_nan_values = observed_data.isnull().mean() * 100
        
        # Check if there are any non-null values
        #if not observed_data.isnull().all():
        if percentage_nan_values < 33:
        
            simulated_nonCP_data = tasmax_values_nonCP[j,:]
            simulated_CP_data = tasmax_values_CP[j,:]

            # Find temperature range on which computing ECDFs (1 bin = 1 °C)
            range_max = round(np.nanmax(observed_data))
            range_min = round(np.nanmin(observed_data))
            temperature_range = np.arange(range_min, range_max + 1)

            # Compute the ECDF
            x_sim_obs, f_sim_obs = ecdf(observed_data)
            x_sim_nonCP, f_sim_nonCP = ecdf(simulated_nonCP_data)
            x_sim_CP, f_sim_CP = ecdf(simulated_CP_data)

            # Create the histograms
            hist_obs, bins_obs = np.histogram(x_sim_obs, bins=np.arange(temperature_range[0], temperature_range[-1] + 1, 1), weights=f_sim_obs, density=True)
            hist_nonCP, bins_nonCP = np.histogram(x_sim_nonCP, bins=np.arange(temperature_range[0], temperature_range[-1] + 1, 1), weights=f_sim_nonCP, density=True)
            hist_CP, bins_CP = np.histogram(x_sim_CP, bins=np.arange(temperature_range[0], temperature_range[-1] + 1, 1), weights=f_sim_CP, density=True)

            # For S_nonCP
            S_nonCP = np.sum(np.minimum(hist_nonCP, hist_obs))

            # For S_CP
            S_CP = np.sum(np.minimum(hist_CP, hist_obs))

            DAV = (S_CP - S_nonCP) / S_nonCP * 100 
            
            if S_nonCP == 0 and S_CP >= 1 :  
                DAV = 100
            elif S_nonCP == 0 and S_CP == 0 :  
                DAV = np.nan
            
        else: 
            print(f"Skipping site {j} due to high percentage of NaN values: {percentage_nan_values}%")
            DAV = np.nan 

        DAV_sites.append(DAV)  
        model_name=MODELS[i]

        # Fill DataFrame columns at each iteration
        df_S_nonCP.at[counter, model_name] = S_nonCP
        df_S_CP.at[counter, model_name] = S_CP
        df_DAV.at[counter, model_name] = DAV

        # Increment the counter for the next iteration
        counter += 1
        

BCCR-AUTH nonCP
BCCR-AUTH CP
Skipping site 1 due to high percentage of NaN values: 52.5%
Skipping site 2 due to high percentage of NaN values: 33.58695652173913%
Skipping site 3 due to high percentage of NaN values: 99.89130434782608%
Skipping site 4 due to high percentage of NaN values: 52.5%
Skipping site 5 due to high percentage of NaN values: 52.5%
Skipping site 6 due to high percentage of NaN values: 99.23913043478261%
Skipping site 7 due to high percentage of NaN values: 33.47826086956522%
Skipping site 8 due to high percentage of NaN values: 33.58695652173913%
Skipping site 9 due to high percentage of NaN values: 99.23913043478261%
Skipping site 10 due to high percentage of NaN values: 98.36956521739131%
Skipping site 11 due to high percentage of NaN values: 33.369565217391305%
Skipping site 12 due to high percentage of NaN values: 52.71739130434783%
Skipping site 13 due to high percentage of NaN values: 52.5%
Skipping site 14 due to high percentage of NaN values: 52.5%
Skipping

BTU CP
Skipping site 1 due to high percentage of NaN values: 52.5%
Skipping site 2 due to high percentage of NaN values: 33.58695652173913%
Skipping site 3 due to high percentage of NaN values: 99.89130434782608%
Skipping site 4 due to high percentage of NaN values: 52.5%
Skipping site 5 due to high percentage of NaN values: 52.5%
Skipping site 6 due to high percentage of NaN values: 99.23913043478261%
Skipping site 7 due to high percentage of NaN values: 33.47826086956522%
Skipping site 8 due to high percentage of NaN values: 33.58695652173913%
Skipping site 9 due to high percentage of NaN values: 99.23913043478261%
Skipping site 10 due to high percentage of NaN values: 98.36956521739131%
Skipping site 11 due to high percentage of NaN values: 33.369565217391305%
Skipping site 12 due to high percentage of NaN values: 52.71739130434783%
Skipping site 13 due to high percentage of NaN values: 52.5%
Skipping site 14 due to high percentage of NaN values: 52.5%
Skipping site 15 due to high p

/home/lorenzosangelantoni/.conda/envs/DEVELOP/lib/python3.10/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges


CMCC CP
Skipping site 1 due to high percentage of NaN values: 52.5%
Skipping site 2 due to high percentage of NaN values: 33.58695652173913%
Skipping site 3 due to high percentage of NaN values: 99.89130434782608%
Skipping site 4 due to high percentage of NaN values: 52.5%
Skipping site 5 due to high percentage of NaN values: 52.5%
Skipping site 6 due to high percentage of NaN values: 99.23913043478261%
Skipping site 7 due to high percentage of NaN values: 33.47826086956522%
Skipping site 8 due to high percentage of NaN values: 33.58695652173913%
Skipping site 9 due to high percentage of NaN values: 99.23913043478261%
Skipping site 10 due to high percentage of NaN values: 98.36956521739131%
Skipping site 11 due to high percentage of NaN values: 33.369565217391305%
Skipping site 12 due to high percentage of NaN values: 52.71739130434783%
Skipping site 13 due to high percentage of NaN values: 52.5%
Skipping site 14 due to high percentage of NaN values: 52.5%
Skipping site 15 due to high 

CNRM CP
Skipping site 1 due to high percentage of NaN values: 52.5%
Skipping site 2 due to high percentage of NaN values: 33.58695652173913%
Skipping site 3 due to high percentage of NaN values: 99.89130434782608%
Skipping site 4 due to high percentage of NaN values: 52.5%
Skipping site 5 due to high percentage of NaN values: 52.5%
Skipping site 6 due to high percentage of NaN values: 99.23913043478261%
Skipping site 7 due to high percentage of NaN values: 33.47826086956522%
Skipping site 8 due to high percentage of NaN values: 33.58695652173913%
Skipping site 9 due to high percentage of NaN values: 99.23913043478261%
Skipping site 10 due to high percentage of NaN values: 98.36956521739131%
Skipping site 11 due to high percentage of NaN values: 33.369565217391305%
Skipping site 12 due to high percentage of NaN values: 52.71739130434783%
Skipping site 13 due to high percentage of NaN values: 52.5%
Skipping site 14 due to high percentage of NaN values: 52.5%
Skipping site 15 due to high 

...

ICTP CP
Skipping site 1 due to high percentage of NaN values: 52.5%
Skipping site 2 due to high percentage of NaN values: 33.58695652173913%
Skipping site 3 due to high percentage of NaN values: 99.89130434782608%
Skipping site 4 due to high percentage of NaN values: 52.5%
Skipping site 5 due to high percentage of NaN values: 52.5%
Skipping site 6 due to high percentage of NaN values: 99.23913043478261%
Skipping site 7 due to high percentage of NaN values: 33.47826086956522%
Skipping site 8 due to high percentage of NaN values: 33.58695652173913%
Skipping site 9 due to high percentage of NaN values: 99.23913043478261%
Skipping site 10 due to high percentage of NaN values: 98.36956521739131%
Skipping site 11 due to high percentage of NaN values: 33.369565217391305%
Skipping site 12 due to high percentage of NaN values: 52.71739130434783%
Skipping site 13 due to high percentage of NaN values: 52.5%
Skipping site 14 due to high percentage of NaN values: 52.5%
Skipping site 15 due to high 

KIT CP
Skipping site 1 due to high percentage of NaN values: 52.5%
Skipping site 2 due to high percentage of NaN values: 33.58695652173913%
Skipping site 3 due to high percentage of NaN values: 99.89130434782608%
Skipping site 4 due to high percentage of NaN values: 52.5%
Skipping site 5 due to high percentage of NaN values: 52.5%
Skipping site 6 due to high percentage of NaN values: 99.23913043478261%
Skipping site 7 due to high percentage of NaN values: 33.47826086956522%
Skipping site 8 due to high percentage of NaN values: 33.58695652173913%
Skipping site 9 due to high percentage of NaN values: 99.23913043478261%
Skipping site 10 due to high percentage of NaN values: 98.36956521739131%
Skipping site 11 due to high percentage of NaN values: 33.369565217391305%
Skipping site 12 due to high percentage of NaN values: 52.71739130434783%
Skipping site 13 due to high percentage of NaN values: 52.5%
Skipping site 14 due to high percentage of NaN values: 52.5%
Skipping site 15 due to high p

KNMI CP
Skipping site 1 due to high percentage of NaN values: 52.5%
Skipping site 2 due to high percentage of NaN values: 33.58695652173913%
Skipping site 3 due to high percentage of NaN values: 99.89130434782608%
Skipping site 4 due to high percentage of NaN values: 52.5%
Skipping site 5 due to high percentage of NaN values: 52.5%
Skipping site 6 due to high percentage of NaN values: 99.23913043478261%
Skipping site 7 due to high percentage of NaN values: 33.47826086956522%
Skipping site 8 due to high percentage of NaN values: 33.58695652173913%
Skipping site 9 due to high percentage of NaN values: 99.23913043478261%
Skipping site 10 due to high percentage of NaN values: 98.36956521739131%
Skipping site 11 due to high percentage of NaN values: 33.369565217391305%
Skipping site 12 due to high percentage of NaN values: 52.71739130434783%
Skipping site 13 due to high percentage of NaN values: 52.5%
Skipping site 14 due to high percentage of NaN values: 52.5%
Skipping site 15 due to high 

UKMO CP
Skipping site 1 due to high percentage of NaN values: 52.5%
Skipping site 2 due to high percentage of NaN values: 33.58695652173913%
Skipping site 3 due to high percentage of NaN values: 99.89130434782608%
Skipping site 4 due to high percentage of NaN values: 52.5%
Skipping site 5 due to high percentage of NaN values: 52.5%
Skipping site 6 due to high percentage of NaN values: 99.23913043478261%
Skipping site 7 due to high percentage of NaN values: 33.47826086956522%
Skipping site 8 due to high percentage of NaN values: 33.58695652173913%
Skipping site 9 due to high percentage of NaN values: 99.23913043478261%
Skipping site 10 due to high percentage of NaN values: 98.36956521739131%
Skipping site 11 due to high percentage of NaN values: 33.369565217391305%
Skipping site 12 due to high percentage of NaN values: 52.71739130434783%
Skipping site 13 due to high percentage of NaN values: 52.5%
Skipping site 14 due to high percentage of NaN values: 52.5%
Skipping site 15 due to high 

JLU CP
Skipping site 1 due to high percentage of NaN values: 52.5%
Skipping site 2 due to high percentage of NaN values: 33.58695652173913%
Skipping site 3 due to high percentage of NaN values: 99.89130434782608%
Skipping site 4 due to high percentage of NaN values: 52.5%
Skipping site 5 due to high percentage of NaN values: 52.5%
Skipping site 6 due to high percentage of NaN values: 99.23913043478261%
Skipping site 7 due to high percentage of NaN values: 33.47826086956522%
Skipping site 8 due to high percentage of NaN values: 33.58695652173913%
Skipping site 9 due to high percentage of NaN values: 99.23913043478261%
Skipping site 10 due to high percentage of NaN values: 98.36956521739131%
Skipping site 11 due to high percentage of NaN values: 33.369565217391305%
Skipping site 12 due to high percentage of NaN values: 52.71739130434783%
Skipping site 13 due to high percentage of NaN values: 52.5%
Skipping site 14 due to high percentage of NaN values: 52.5%
Skipping site 15 due to high p

In [11]:
# Iterate over each model for scatterplots
for model_name in MODELS:
    # Select DAV values for the current model
    DAV_model = df_DAV[model_name]

    fig, ax = plt.subplots(subplot_kw={'projection': ccrs.PlateCarree()})
    ax.coastlines()

    # Add land-sea mask first
    #land_mask = cfeature.NaturalEarthFeature('physical', 'land', '10m', edgecolor='face', facecolor=cfeature.COLORS['land'])
    land_mask = cfeature.NaturalEarthFeature('physical', 'land', '10m', edgecolor='face', facecolor='darkgrey')
    ax.add_feature(land_mask, zorder=0)  # Set zorder to 0 to ensure it's plotted below the scatter points
    ax.set_extent([lon_range.start, 17, lat_range.start, lat_range.stop])  # Limit to 17 degrees west
 
    # Add land and state borders
    ax.add_feature(cfeature.BORDERS, linestyle='-')
    ax.add_feature(cfeature.COASTLINE)

    #gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=1, color='gray', alpha=0.5, linestyle=None)
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=0, color='gray', alpha=0.5, linestyle='-')
    gl.xlabels_top = False
    gl.ylabels_right = False
    gl.xformatter = LongitudeFormatter(degree_symbol='°', dateline_direction_label=True)
    gl.yformatter = LatitudeFormatter(degree_symbol='°')

    # Set gridline intervals to every 2 degrees
    gl.xlocator = plt.MaxNLocator(5)
    gl.ylocator = plt.MaxNLocator(5)
    
    # Scatter plot for DAV values without transparency
    sc = ax.scatter(site_lons, site_lats, c=DAV_model, cmap=sns.color_palette("vlag", as_cmap=True), s=30, edgecolors='k', vmin=-100, vmax=100, alpha=1.0)
    
    # Add colorbar
    cbar = plt.colorbar(sc, orientation='vertical', label='% DAV', shrink=0.65)

    plt.title(f'DAV {model_name}')
    plt.savefig(f'DAV_{model_name}.png')
    
    print(f'{model_name} done')
   
    
# Calculate ensemble mean across models
df_DAV['Ensemble_Mean'] = df_DAV[MODELS].mean(axis=1)

# Set values not finite to NaN
df_DAV['Ensemble_Mean'] = np.where(np.isfinite(df_DAV['Ensemble_Mean']), df_DAV['Ensemble_Mean'], np.nan)

# Calculate the percentage of sites with positive DAV
positive_dav_percentage = (df_DAV['Ensemble_Mean'] > 0).sum() / df_DAV['Ensemble_Mean'].notnull().sum() * 100
# Calculate overall mean DAV across all sites
mean_DAV_all_sites = np.nanmean(df_DAV['Ensemble_Mean'])


fig, ax = plt.subplots(subplot_kw={'projection': ccrs.PlateCarree()})
ax.coastlines()
land_mask = cfeature.NaturalEarthFeature('physical', 'land', '10m', edgecolor='face', facecolor='darkgrey')
ax.add_feature(land_mask, zorder=0)  # Set zorder to 0 to ensure it's plotted below the scatter points
ax.set_extent([lon_range.start, 17, lat_range.start, lat_range.stop])  # Limit to 17 degrees west
ax.add_feature(cfeature.BORDERS, linestyle='-')
ax.add_feature(cfeature.COASTLINE)

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=0, color='gray', alpha=0.5, linestyle='-')
gl.xlabels_top = False
gl.ylabels_right = False
gl.xformatter = LongitudeFormatter(degree_symbol='°', dateline_direction_label=True)
gl.yformatter = LatitudeFormatter(degree_symbol='°')
gl.xlocator = plt.MaxNLocator(5)
gl.ylocator = plt.MaxNLocator(5)

sc = ax.scatter(site_lons, site_lats, c=df_DAV['Ensemble_Mean'], cmap=sns.color_palette("vlag", as_cmap=True), s=30, edgecolors='k', vmin=-100, vmax=100, alpha=1)

cbar = plt.colorbar(sc, orientation='vertical', label='% DAV', shrink=0.65)

# Create a multiline text
info_text = f"DAV > 0: {positive_dav_percentage:.0f}%\nMean DAV: {mean_DAV_all_sites:.0f}"
# Add the text to the lower-left part of the map
fig.text(0.2, 0.05, info_text, ha='left', va='bottom', transform=ax.transAxes, fontsize=10,bbox=dict(facecolor='white', alpha=0.7),color='red')

plt.title(f'DAV ENSEMBLE')
plt.savefig(f'DAV_ENS.png')
print(f'ENS done')

/home/lorenzosangelantoni/.conda/envs/DEVELOP/lib/python3.10/site-packages/cartopy/mpl/gridliner.py:451: UserWarning: The .xlabels_top attribute is deprecated. Please use .top_labels to toggle visibility instead.
  warnings.warn('The .xlabels_top attribute is deprecated. Please '
/home/lorenzosangelantoni/.conda/envs/DEVELOP/lib/python3.10/site-packages/cartopy/mpl/gridliner.py:487: UserWarning: The .ylabels_right attribute is deprecated. Please use .right_labels to toggle visibility instead.
  warnings.warn('The .ylabels_right attribute is deprecated. Please '


BCCR-AUTH done
BTU done
CMCC done
CNRM done
ETHZ done
FZJ-IDL done


/tmp/ipykernel_68343/2399450632.py:6: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(subplot_kw={'projection': ccrs.PlateCarree()})


HCLIM done
ICTP done
KIT done
KNMI done
UKMO done
JLU done


/home/lorenzosangelantoni/.conda/envs/DEVELOP/lib/python3.10/site-packages/cartopy/mpl/gridliner.py:451: UserWarning: The .xlabels_top attribute is deprecated. Please use .top_labels to toggle visibility instead.
  warnings.warn('The .xlabels_top attribute is deprecated. Please '
/home/lorenzosangelantoni/.conda/envs/DEVELOP/lib/python3.10/site-packages/cartopy/mpl/gridliner.py:487: UserWarning: The .ylabels_right attribute is deprecated. Please use .right_labels to toggle visibility instead.
  warnings.warn('The .ylabels_right attribute is deprecated. Please '


ENS done


### DAV over the 90th percentile 

In [6]:
DAV_sites = []

# Calculate the ECDF
def ecdf(data):
    sorted_data = np.sort(data)
    n = len(data)
    y = np.arange(1, n + 1) / n
    return sorted_data, y

# Assuming you have a counter defined somewhere
counter = 0

for j in range(0, len(site_lons)):

    observed_data = filtered_dataframes[j]['temperature']

    # Check if there are any non-null values
    if not observed_data.isnull().all():
        simulated_nonCP_data = tasmax_values_nonCP[j, :]
        simulated_CP_data = tasmax_values_CP[j, :]

        # Find temperature range on which computing ECDFs (1 bin = 1 °C)
        range_max = round(np.nanmax(observed_data))
        range_min = round(np.nanmin(observed_data))
        temperature_range = np.arange(range_min, range_max + 1)

        # Compute the ECDF
        x_sim_obs, f_sim_obs = ecdf(observed_data)
        x_sim_nonCP, f_sim_nonCP = ecdf(simulated_nonCP_data)
        x_sim_CP, f_sim_CP = ecdf(simulated_CP_data)

        # Calculate the 90th percentile for observed data
        percentile_90_observed = np.percentile(observed_data, 90)

        # Filter values equal to or higher than the 90th percentile
        x_sim_obs_90 = x_sim_obs[x_sim_obs >= percentile_90_observed]
        f_sim_obs_90 = f_sim_obs[x_sim_obs >= percentile_90_observed]

        # Create the histograms for filtered data
        hist_obs_90, bins_obs_90 = np.histogram(x_sim_obs_90, bins=np.arange(temperature_range[0], temperature_range[-1] + 1, 1),
                                               weights=f_sim_obs_90, density=True)
        hist_nonCP, bins_nonCP = np.histogram(x_sim_nonCP, bins=np.arange(temperature_range[0], temperature_range[-1] + 1, 1),
                                              weights=f_sim_nonCP, density=True)
        hist_CP, bins_CP = np.histogram(x_sim_CP, bins=np.arange(temperature_range[0], temperature_range[-1] + 1, 1),
                                        weights=f_sim_CP, density=True)

        # For S_nonCP
        S_nonCP = np.sum(np.minimum(hist_nonCP, hist_obs_90))

        # For S_CP
        S_CP = np.sum(np.minimum(hist_CP, hist_obs_90))

        DAV = (S_CP - S_nonCP) / S_nonCP * 100

        if S_nonCP == 0 and S_CP >= 1:
            DAV = 100
        elif S_nonCP == 0 and S_CP == 0:
            DAV = np.nan

    else:
        DAV = np.nan

    DAV_sites.append(DAV)
    model_name = MODELS[i]

    # Fill DataFrame columns at each iteration
    df_S_nonCP.at[counter, model_name] = S_nonCP
    df_S_CP.at[counter, model_name] = S_CP
    df_DAV.at[counter, model_name] = DAV

    # Increment the counter for the next iteration
    counter += 1


/home/lorenzosangelantoni/.conda/envs/DEVELOP/lib/python3.10/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges
/tmp/ipykernel_63915/2224755549.py:53: RuntimeWarning: invalid value encountered in double_scalars
  DAV = (S_CP - S_nonCP) / S_nonCP * 100


In [9]:
DAV_model

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
      ... 
635    NaN
636    NaN
637    NaN
638    NaN
639    NaN
Name: JLU, Length: 640, dtype: object

### Plot DAV over the 90th percentile 

In [10]:
# Iterate over each model for scatterplots
for model_name in MODELS[0:-1]:
    # Select DAV values for the current model
    DAV_model = df_DAV[model_name]

    fig, ax = plt.subplots(subplot_kw={'projection': ccrs.PlateCarree()})
    ax.coastlines()

    # Add land-sea mask first
    #land_mask = cfeature.NaturalEarthFeature('physical', 'land', '10m', edgecolor='face', facecolor=cfeature.COLORS['land'])
    land_mask = cfeature.NaturalEarthFeature('physical', 'land', '10m', edgecolor='face', facecolor='darkgrey')
    ax.add_feature(land_mask, zorder=0)  # Set zorder to 0 to ensure it's plotted below the scatter points
    ax.set_extent([lon_range.start, 17, lat_range.start, lat_range.stop])  # Limit to 17 degrees west
 
    # Add land and state borders
    ax.add_feature(cfeature.BORDERS, linestyle='-')
    ax.add_feature(cfeature.COASTLINE)

    #gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=1, color='gray', alpha=0.5, linestyle=None)
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=0, color='gray', alpha=0.5, linestyle='-')
    gl.xlabels_top = False
    gl.ylabels_right = False
    gl.xformatter = LongitudeFormatter(degree_symbol='°', dateline_direction_label=True)
    gl.yformatter = LatitudeFormatter(degree_symbol='°')

    # Set gridline intervals to every 2 degrees
    gl.xlocator = plt.MaxNLocator(5)
    gl.ylocator = plt.MaxNLocator(5)
    
    # Scatter plot for DAV values without transparency
    sc = ax.scatter(site_lons, site_lats, c=DAV_model, cmap=sns.color_palette("vlag", as_cmap=True), s=30, edgecolors='k', vmin=-100, vmax=100, alpha=1.0)
    
    # Add colorbar
    cbar = plt.colorbar(sc, orientation='vertical', label='% DAV', shrink=0.65)

    plt.title(f'DAV {model_name}')
    plt.savefig(f'DAV_{model_name}.png')
    
    print(f'{model_name} done')
   
    
# Calculate ensemble mean across models
df_DAV['Ensemble_Mean'] = df_DAV[MODELS].mean(axis=1)

# Set values not finite to NaN
df_DAV['Ensemble_Mean'] = np.where(np.isfinite(df_DAV['Ensemble_Mean']), df_DAV['Ensemble_Mean'], np.nan)

# Calculate the percentage of sites with positive DAV
positive_dav_percentage = (df_DAV['Ensemble_Mean'] > 0).sum() / df_DAV['Ensemble_Mean'].notnull().sum() * 100
# Calculate overall mean DAV across all sites
mean_DAV_all_sites = np.nanmean(df_DAV['Ensemble_Mean'])


fig, ax = plt.subplots(subplot_kw={'projection': ccrs.PlateCarree()})
ax.coastlines()
land_mask = cfeature.NaturalEarthFeature('physical', 'land', '10m', edgecolor='face', facecolor='darkgrey')
ax.add_feature(land_mask, zorder=0)  # Set zorder to 0 to ensure it's plotted below the scatter points
ax.set_extent([lon_range.start, 17, lat_range.start, lat_range.stop])  # Limit to 17 degrees west
ax.add_feature(cfeature.BORDERS, linestyle='-')
ax.add_feature(cfeature.COASTLINE)

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=0, color='gray', alpha=0.5, linestyle='-')
gl.xlabels_top = False
gl.ylabels_right = False
gl.xformatter = LongitudeFormatter(degree_symbol='°', dateline_direction_label=True)
gl.yformatter = LatitudeFormatter(degree_symbol='°')
gl.xlocator = plt.MaxNLocator(5)
gl.ylocator = plt.MaxNLocator(5)

sc = ax.scatter(site_lons, site_lats, c=df_DAV['Ensemble_Mean'], cmap=sns.color_palette("vlag", as_cmap=True), s=30, edgecolors='k', vmin=-100, vmax=100, alpha=1)

cbar = plt.colorbar(sc, orientation='vertical', label='% DAV', shrink=0.65)

# Create a multiline text
info_text = f"DAV > 0: {positive_dav_percentage:.0f}%\nMean DAV: {mean_DAV_all_sites:.0f}"
# Add the text to the lower-left part of the map
fig.text(0.2, 0.05, info_text, ha='left', va='bottom', transform=ax.transAxes, fontsize=10,bbox=dict(facecolor='white', alpha=0.7),color='red')

plt.title(f'DAV ENSEMBLE')
plt.savefig(f'DAV_ENS_lt90p.png')
print(f'ENS done')

BCCR-AUTH done
BTU done
CMCC done
CNRM done
ETHZ done
FZJ-IDL done
HCLIM done
ICTP done
KIT done
KNMI done
UKMO done


/home/lorenzosangelantoni/.conda/envs/DEVELOP/lib/python3.10/site-packages/cartopy/mpl/gridliner.py:451: UserWarning: The .xlabels_top attribute is deprecated. Please use .top_labels to toggle visibility instead.
  warnings.warn('The .xlabels_top attribute is deprecated. Please '
/home/lorenzosangelantoni/.conda/envs/DEVELOP/lib/python3.10/site-packages/cartopy/mpl/gridliner.py:487: UserWarning: The .ylabels_right attribute is deprecated. Please use .right_labels to toggle visibility instead.
  warnings.warn('The .ylabels_right attribute is deprecated. Please '


ENS done


### Violin plot

In [235]:
models_to_compare = ['BCCR-AUTH', 'BTU', 'CMCC', 'CNRM',
                     'ETHZ', 'FZJ-IDL', 'HCLIM', 'ICTP', 'KIT', 'KNMI', 'UKMO','JLU']

# Create a list to store the combined dataframes for each model
combined_dfs = []

# Iterate over the models and create the combined dataframe
for model_name in models_to_compare:
    # Extract data for the chosen model
    data_S_nonCP = df_S_nonCP[model_name]
    data_S_CP = df_S_CP[model_name]
    
    if model_name == 'JLU' : 
        data_S_nonCP = df_S_nonCP['KIT']
        data_S_CP = df_S_CP['JLU']
    
    # Convert the data to numeric, replacing non-numeric values with NaN
    data_S_nonCP = pd.to_numeric(data_S_nonCP, errors='coerce')
    data_S_CP = pd.to_numeric(data_S_CP, errors='coerce')

    # Create a DataFrame for the chosen model
    df_model_nonCP = pd.DataFrame({
        'Resolution': ['S_nonCP'] * len(data_S_nonCP),
        'Temperature': data_S_nonCP
    })

    df_model_CP = pd.DataFrame({
        'Resolution': ['S_CP'] * len(data_S_CP),
        'Temperature': data_S_CP
    })

    # Combine the dataframes for S_nonCP and S_CP for the current model
    df_combined_model = pd.concat([df_model_nonCP, df_model_CP])

    # Add the model name as a column
    df_combined_model['Model'] = model_name

    # Append the combined dataframe to the list
    combined_dfs.append(df_combined_model)

# Combine all the dataframes into a single dataframe
df_combined = pd.concat(combined_dfs)

# Create a split violin plot
plt.figure(figsize=(20, 6))
ax = sns.violinplot(data=df_combined, x="Model", y="Temperature", hue="Resolution", split=True, inner="quart", palette="muted")

ax.set_xticklabels(ax.get_xticklabels(), fontsize=12)  # Adjust the font size for x-axis tick labels
ax.set_yticklabels(ax.get_yticklabels(), fontsize=12)  # Adjust the font size for y-axis tick labelsplt.title('S score')

plt.xlabel('Model', fontsize=14)
plt.ylabel('S score', fontsize=14)
plt.legend(loc='upper right',fontsize=12)
plt.savefig('violin.png')



/tmp/ipykernel_63086/3326954295.py:49: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(ax.get_yticklabels(), fontsize=12)  # Adjust the font size for y-axis tick labelsplt.title('S score')


In [219]:
df_S_nonCP

,BCCR-AUTH,BTU,CMCC,CNRM,ETHZ,FZJ-IDL,HCLIM,ICTP,KIT,KNMI,UKMO
0,0.688406,0.818576,0.791676,0.894299,0.629464,0.64483,0.788572,0.875045,0.724172,0.722454,0.817578
1,0.777112,0.528656,0.53845,0.707599,0.454068,0.519514,0.561862,0.634497,0.537923,0.492995,0.84339
2,0.699444,0.553965,0.57446,0.684527,0.418246,0.561074,0.593544,0.700226,0.581994,0.553481,0.848525
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.390579,0.783267,0.828257,0.71819,0.753882,0.777277,0.811965,0.478615,0.809732,0.782232,0.459811
...,...,...,...,...,...,...,...,...,...,...,...
635,0.725593,0.744482,0.822589,0.751715,0.720233,0.544219,0.630327,0.129929,0.740147,0.641198,0.833147
636,0.680998,0.700868,0.627122,0.6378,0.506018,0.37429,0.490043,0.204093,0.663747,0.329872,0.561274
637,0.828658,0.669434,0.59831,0.594155,0.580978,0.417527,0.61049,0.171661,0.715897,0.466832,0.712328
638,0.676817,0.68927,0.644061,0.643498,0.658579,0.617875,0.685564,0.440743,0.647561,0.655902,0.655866


In [210]:
# Initialize lists to store values for each model
all_S_nonCP = []
all_S_CP = []
all_DAV = []
model_names = []

# Initialize DataFrames to store values for each model
df_S_nonCP = pd.DataFrame(index=range(0,len(site_lons)), columns=MODELS)
df_S_CP = pd.DataFrame(index=range(0,len(site_lons)), columns=MODELS)
df_DAV = pd.DataFrame(index=range(0,len(site_lons)), columns=MODELS)

SIM_DIR_nonCP = '/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/FILES_nonCP/tasmax/historical'
SIM_DIR_CP = '/work3/lorenzosangelantoni/scratch75/gcm_driven_experiment_from_nird/FILES_CP/tasmax/historical'

for i in range(0,1) : 
    
    # Construct the full path to the file
    file_path = os.path.join(f'{SIM_DIR_nonCP}/{filename_nonCP[i]}')
        
    print(f'{MODELS[i]} nonCP')
        
    nc = Dataset(file_path)
    lon_model = nc.variables['lon'][:]
    lat_model = nc.variables['lat'][:]
    tasmax = nc.variables['tasmax'][:]
    
    # Reference site coordinates
    reference_coords = list(zip(site_lons, site_lats))
    
    # Flatten the lat and lon arrays from the model data
    model_lon_flat = lon_model.flatten()
    model_lat_flat = lat_model.flatten()

    # Create a KDTree from the flattened lon and lat arrays
    tree = cKDTree(list(zip(model_lon_flat, model_lat_flat)))

    # Query the KDTree to find the nearest neighbors for each reference site
    nearest_indices = []

    for lon, lat in reference_coords:
        dist, idx = tree.query([lon, lat])
        
        # Check if the index is within bounds
        if 0 <= idx < len(model_lon_flat):
            nearest_indices.append(idx)
        else:
            # Handle the case when the index is out of bounds
            nearest_indices.append(np.nan)

    # Extract the lon and lat values for the nearest grid nodes
    nearest_lon = [model_lon_flat[int(idx)] if not np.isnan(idx) else np.nan for idx in nearest_indices]
    nearest_lat = [model_lat_flat[int(idx)] if not np.isnan(idx) else np.nan for idx in nearest_indices]

    # Reshape the tasmax_data variable to have a flattened lat and lon dimension
    tasmax_data_flat = tasmax.reshape(tasmax.shape[0], -1)

    # Initialize an empty list to store tasmax values for each reference site
    tasmax_values = []

    # Extract tasmax values for each reference site
    for idx in nearest_indices:
        if not np.isnan(idx):
            tasmax_values.append(tasmax_data_flat[:, int(idx)])
        else:
            # Handle the case when the index is NaN
            tasmax_values.append(np.full(tasmax.shape[0], np.nan))

    # Convert the list of tasmax values to a NumPy array
    tasmax_values_nonCP = np.array(tasmax_values)-273.16

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
    
    del(tasmax_values)
    
    file_path = os.path.join(f'{SIM_DIR_CP}/{filename_CP[i]}')
        
    print(f'{MODELS[i]} CP')
        
    nc = Dataset(file_path)
    lon_model = nc.variables['lon'][:]
    lat_model = nc.variables['lat'][:]
    tasmax = nc.variables['tasmax'][:]
    
    # Reference site coordinates
    reference_coords = list(zip(site_lons, site_lats))
    
    # Flatten the lat and lon arrays from the model data
    model_lon_flat = lon_model.flatten()
    model_lat_flat = lat_model.flatten()

    # Create a KDTree from the flattened lon and lat arrays
    tree = cKDTree(list(zip(model_lon_flat, model_lat_flat)))

    # Query the KDTree to find the nearest neighbors for each reference site
    nearest_indices = []

    for lon, lat in reference_coords:
        dist, idx = tree.query([lon, lat])
        
        # Check if the index is within bounds
        if 0 <= idx < len(model_lon_flat):
            nearest_indices.append(idx)
        else:
            # Handle the case when the index is out of bounds
            nearest_indices.append(np.nan)

    # Extract the lon and lat values for the nearest grid nodes
    nearest_lon = [model_lon_flat[int(idx)] if not np.isnan(idx) else np.nan for idx in nearest_indices]
    nearest_lat = [model_lat_flat[int(idx)] if not np.isnan(idx) else np.nan for idx in nearest_indices]

    # Reshape the tasmax_data variable to have a flattened lat and lon dimension
    tasmax_data_flat = tasmax.reshape(tasmax.shape[0], -1)

    # Initialize an empty list to store tasmax values for each reference site
    tasmax_values = []

    # Extract tasmax values for each reference site
    for idx in nearest_indices:
        if not np.isnan(idx):
            tasmax_values.append(tasmax_data_flat[:, int(idx)])
        else:
            # Handle the case when the index is NaN
            tasmax_values.append(np.full(tasmax.shape[0], np.nan))

    # Convert the list of tasmax values to a NumPy array
    tasmax_values_CP = np.array(tasmax_values)-273.16

    
# # # #  # # # # # # # # # # # # # # # # # # # # # DAV # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

    DAV_sites=[]

    # Calculate the ECDF
    def ecdf(data):
        sorted_data = np.sort(data)
        n = len(data)
        y = np.arange(1, n + 1) / n
        return sorted_data, y
    
    # Assuming you have a counter defined somewhere
    counter = 0
    for j in range(0,1) : 

        observed_data = filtered_dataframes[j]['temperature']

        # Check if there are any non-null values
        if not observed_data.isnull().all():
            simulated_nonCP_data = tasmax_values_nonCP[j,:]
            simulated_CP_data = tasmax_values_CP[j,:]

            # Find temperature range on which computing ECDFs (1 bin = 1 °C)
            range_max = round(np.nanmax(observed_data))
            range_min = round(np.nanmin(observed_data))
            temperature_range = np.arange(range_min, range_max + 1)

            # Compute the ECDF
            x_sim_obs, f_sim_obs = ecdf(observed_data)
            x_sim_nonCP, f_sim_nonCP = ecdf(simulated_nonCP_data)
            x_sim_CP, f_sim_CP = ecdf(simulated_CP_data)

            # Create the histograms
            hist_obs, bins_obs = np.histogram(x_sim_obs,bins=np.arange(temperature_range[0], temperature_range[-1] + 1, 1), weights=f_sim_obs,density=True)
            hist_nonCP, bins_nonCP = np.histogram(x_sim_nonCP, bins=np.arange(temperature_range[0], temperature_range[-1] + 1, 1), weights=f_sim_nonCP, density=True)
            hist_CP, bins_CP = np.histogram(x_sim_CP, bins=np.arange(temperature_range[0], temperature_range[-1] + 1, 1), weights=f_sim_CP, density=True)

            # For S_nonCP
            S_nonCP = np.sum(np.minimum(hist_nonCP, hist_obs))

            # For S_CP
            S_CP = np.sum(np.minimum(hist_CP, hist_obs))

            DAV = (S_CP - S_nonCP) / S_nonCP * 100 
            
            if S_nonCP == 0 and S_CP >= 1 :  
                DAV = 100
            elif S_nonCP == 0 and S_CP == 0 :  
                DAV = np.nan
            
        else: 
            DAV = np.nan 

        DAV_sites.append(DAV)  
        model_name=MODELS[i]

        # Fill DataFrame columns at each iteration
        df_S_nonCP.at[counter, model_name] = S_nonCP
        df_S_CP.at[counter, model_name] = S_CP
        df_DAV.at[counter, model_name] = DAV

        # Increment the counter for the next iteration
        counter += 1
        

BCCR-AUTH nonCP
BCCR-AUTH CP


In [215]:
S_nonCP

0.6884059864807937

In [212]:
# Plot ECDFs
plt.figure(figsize=(10, 6))

# Plot observed data ECDF
#plt.plot(hist_obs, label='Observations', marker='o', linestyle='-', color='blue')
# Plot simulated data ECDF
#plt.plot(hist_nonCP, label='Simulations', marker='o', linestyle='-', color='orange')

# Add histograms for observed and simulated data
plt.hist(hist_obs, alpha=0.5, color='blue', density=True, edgecolor='black',label='Observations')
#plt.hist(bins_obs, hist_nonCP, alpha=0.5, color='orange', density=True, edgecolor='black',label='Simulations')

plt.title('Empirical Cumulative Distribution Functions with Histograms')
plt.xlabel('Data Values')
plt.ylabel('Cumulative Probability')
plt.legend()
plt.grid(True)
plt.savefig('ecdfs.png')

In [ ]:
from statsmodels.distributions.empirical_distribution import ECDF
ecdf_observed = ECDF(observed_data)